In [4]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import root

## Non-closed-form solution
The non-closed-form solution for probability of default (PD) in the Merton model is given by:

$$ E = V \Phi(d_1) - D e^{-rT}\Phi(d_2) $$
$$ \sigma_E = \frac{V}{E} \Phi(d_1)\sigma_V $$

where:
- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution
- $ d_1 = \frac{\ln\frac{V}{D} + (r + 0.5{\sigma_V}^2)T}{\sigma_V\sqrt{T}} $
- $ d_2 = d_1 - \sigma_V\sqrt{T} $
- $ E $ is the equity value
- $ \sigma_E $ is the equity volatility 
- $ V $ is the asset value. This is unknown and solved for.
- $ \sigma_V $ is the asset volatility. This is unknown and solved for.
- $ D $ is the debt face value.
- $ r $ is the risk-free rate.
- $ T $ is the time to maturity.

These two functions are used to solve for $ V $ and $ \sigma_V $. Once that is done, the following equation is used to find probability of default:

$$ PD = \Phi\left( \frac{\ln\left(\frac{D}{V}\right) - \left( r - \frac{1}{2} \sigma_V^2 \right) T}{\sigma_V \sqrt{T}} \right)$$



where:

- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution  
- $ V $ is the value of the firm's assets  
- $ D $ is the face value of debt 
- $ r $ is the risk-free interest rate  
- $ \sigma_V $ is the volatility of the firm's assets  
- $ T $ is the time to maturity




In [5]:
def compute_merton_pd(E, sigma_E, D, r, T=1.0, verbose=False):    

    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            return 1e10, 1e10  # large penalty for invalid values
        d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
        d2 = d1 - sigma_V * np.sqrt(T)
        eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
        eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
        return [eq1, eq2]

    V0 = E + D
    sigma_V0 = sigma_E
    result = root(equations, [V0, sigma_V0], method='hybr')

    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        if verbose:
            print("Failure for firm:", E, sigma_E, D, r)
            print("Message:", result.message)
        return np.nan

In [6]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'],
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_35748/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_35748/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 48621473472.80335 1.7631814439192062 127628500000.0 0.028900000000000002
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 46370479330.54394 1.7632036405195546 127628500000.0 0.029500000000000002
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 45019882845.188286 1.7632586329150917 127628500000.0 0.029500000000000002
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 46820678158.99582 1.7641001625955732 127628500000.0 0.0305
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 46820678158.99582 1.7641646854030906 127628500000.0 0.030699999999999998
Message: The iteration is not making good progress, as measured by the 
 improvement from the last

In [7]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output.csv", index=False)

In [8]:
df['pd_valid'].value_counts()

pd_valid
True     439406
False      1644
Name: count, dtype: int64

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno   tic                          conm         PRC  \
23870   2009-02-03   66800   AIG  AMERICAN INTERNATIONAL GROUP   18.075314   
23871   2009-02-04   66800   AIG  AMERICAN INTERNATIONAL GROUP   17.238494   
23872   2009-02-05   66800   AIG  AMERICAN INTERNATIONAL GROUP   16.736402   
23873   2009-02-06   66800   AIG  AMERICAN INTERNATIONAL GROUP   17.405858   
23874   2009-02-09   66800   AIG  AMERICAN INTERNATIONAL GROUP   17.405858   
23876   2009-02-11   66800   AIG  AMERICAN INTERNATIONAL GROUP   16.066946   
23877   2009-02-12   66800   AIG  AMERICAN INTERNATIONAL GROUP   15.062762   
23878   2009-02-13   66800   AIG  AMERICAN INTERNATIONAL GROUP   14.225941   
23879   2009-02-17   66800   AIG  AMERICAN INTERNATIONAL GROUP   13.054393   
23880   2009-02-18   66800   AIG  AMERICAN INTERNATIONAL GROUP   12.217573   
23881   2009-02-19   66800   AIG  AMERICAN INTERNATIONAL GROUP    9.874477   
23882   2009-02-20   66800   AIG  AMERICAN INTERNATIONAL GROUP  

In [10]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    1644.000000
mean        0.340575
std         0.154343
min         0.018998
25%         0.192402
50%         0.400062
75%         0.483374
max         0.575617
Name: leverage, dtype: float64

count    1644.000000
mean        0.881973
std         0.605007
min         0.147026
25%         0.309050
50%         0.804703
75%         1.413659
max         2.216216
Name: equity_volatility, dtype: float64


In [11]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    441050.000000
mean          0.156678
std           0.099050
min           0.000000
25%           0.089657
50%           0.142806
75%           0.206324
max           0.575617
Name: leverage, dtype: float64

count    441050.000000
mean          2.206908
std          14.811601
min           0.090416
25%           0.187856
50%           0.239566
75%           0.319094
max         116.571964
Name: equity_volatility, dtype: float64
